In [1]:
import csv
import numpy as np

In [2]:
numtrain = 0.8
datasetRed = list([])
with open(r'C:\Users\Kittichai\test409\cs409\project\winequality-red.csv' ,mode='r') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=';')
    line_count = 0
    for row in csv_reader:
        if line_count != 0:
            datasetRed.append(row)
            #datasetRed.append(np.array(row).astype(np.float))
        line_count += 1

numtrainRed = int(numtrain * len(datasetRed))
numtestRed = len(datasetRed) - numtrainRed 

data_tran_red = list([])
aws_tran_red = np.ones([numtrainRed,1])

data_test_red = list([])
aws_test_red = np.ones([numtestRed,1])


for i in range(numtrainRed):
    tt = datasetRed.pop(np.random.randint(len(datasetRed)))
    data_tran_red.append(tt[0:11])
    aws_tran_red[i] = tt[11]

for i in range(len(datasetRed)):
    data_test_red.append(datasetRed[i][0:11])
    aws_test_red[i] = datasetRed[i][11]

data_tran_red = np.array(data_tran_red).astype(np.float)
data_test_red = np.array(data_test_red).astype(np.float)


###################################################################### 
numtrain = 0.8
datasetWhile = list([])
with open(r'C:\Users\Kittichai\test409\cs409\project\winequality-white.csv' ,mode='r') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=';')
    line_count = 0
    for row in csv_reader:
        if line_count != 0:
            datasetWhile.append(row)
            #datasetRed.append(np.array(row).astype(np.float))
        line_count += 1

numtrainWhile = int(numtrain * len(datasetWhile))
numtestWhile = len(datasetWhile) - numtrainWhile 

data_tran_While = list([])
aws_tran_While = np.ones([numtrainWhile,1])

data_test_While = list([])
aws_test_While = np.ones([numtestWhile,1])


for i in range(numtrainWhile):
    tt = datasetWhile.pop(np.random.randint(len(datasetWhile)))
    data_tran_While.append(tt[0:11])
    aws_tran_While[i] = tt[11]
for i in range(len(datasetWhile)):
    data_test_While.append(datasetWhile[i][0:11])
    aws_test_While[i] = datasetWhile[i][11]
    
    

data_tran_While = np.array(data_tran_While).astype(np.float)
data_test_While = np.array(data_test_While).astype(np.float)


In [3]:
#tanh(x)
def g(x):    
    return (np.exp(x) - np.exp(-x))/(np.exp(x) + np.exp(-x)) 

def grad_g(g):
    return 1-g*g


In [4]:
#random init weight and bias
np.random.seed(1)
N0=6 #first layer 6 nodes
N1=3 #second layer 3 nodes
N2=1
eta=0.02

a0=np.ones((len(data_tran_red),12)) #[Nx12]

a0[:,0:11]=data_tran_red

w0 = np.random.random((11,N0)) # 9 inputs x 6 nodes
b0 = np.random.random((1,N0)) # 1 bias x 6 nodes
wb0=np.concatenate((w0, b0), axis=0)

a1=np.ones((len(data_tran_red),N0+1))
w1 = np.random.random((N0,N1)) # 6 inputs x 3 nodes
b1 = np.random.random((1,N1)) # 1 bias x 3 nodes
wb1=np.concatenate((w1, b1), axis=0)

a2=np.ones((len(data_tran_red),N1+1))
w2 = np.random.random((N1,N2)) # 3 input * 1 node
b2 = np.random.random((1,N2)) # 1 bias * 1 node
wb2=np.concatenate((w2, b2), axis=0)

a3 = np.ones((len(data_tran_red),N2))

for i in range(200000):
    #forward x=a0, a2=yp
    z0=np.dot(a0,wb0)
    a1[:,:-1]=g(z0)
    #a1=np.concatenate([g(z0),np.ones([a0.shape[0],1])], axis=1)
    
    z1=np.dot(a1,wb1)
    a2[:,:-1]=g(z1)
   
    z2=np.dot(a2,wb2)
    a3=g(z2)
    
    #backward
    
    d_a3=aws_tran_red-a3
    d_z2=d_a3*grad_g(a3)
    wb2 += eta*np.dot(a2.T,d_z2)
    
    d_a2=np.dot(d_z2,wb2.T)
    d_z1=d_a2*grad_g(a2)
    wb1 += eta*np.dot(a1.T,d_z1)[:,:-1]

    d_a1=np.dot(d_z1[:,:-1],wb1.T)
    d_z0=d_a1*grad_g(a1)
    wb0 += eta*np.dot(a0.T,d_z0[:,:-1])


MAE = (np.sum(np.absolute(d_a3)))/ len(d_a3)
print("MAE: {}".format(MAE))

MSE=0.5*np.sum(np.square(d_a3))
print("MSE: {}".format(MSE))

RMSE= np.sum(np.power(d_a3,2)) / len(d_a3)
print("RMSE: {}".format(RMSE))

print("========== TEST =============")     
a0=np.ones((len(data_test_red),12)) #[Nx10]

a0[:,0:11] = data_test_red 

a1=np.ones((len(data_test_red),N0+1))
a2=np.ones((len(data_test_red),N1+1))

#forward x=a0, a2=yp
z0=np.dot(a0,wb0)
a1[:,:-1]=g(z0)
#a1=np.concatenate([g(z0),np.ones([a0.shape[0],1])], axis=1)
    
z1=np.dot(a1,wb1)
a2[:,:-1]=g(z1)
   
z2=np.dot(a2,wb2)
a3=g(z2)

    
d_a3=aws_test_red-a3
MAE = (np.sum(np.absolute(d_a3)))/ len(d_a3)
print("MAE: {}".format(MAE))

MSE=0.5*np.sum(np.square(d_a3))
print("MSE: {}".format(MSE))

RMSE= np.sum(np.power(d_a3,2)) / len(d_a3)
print("RMSE: {}".format(RMSE))


MAE: 0.4700547302580141
MSE: 158.6
RMSE: 0.24800625488663017
========== TEST =============
MAE: 0.48375
MSE: 40.96
RMSE: 0.256


In [5]:
np.random.seed(1)
N0=6 #first layer 6 nodes
N1=3 #second layer 3 nodes
N2=1
eta=0.02

a0=np.ones((len(data_tran_While),12)) #[Nx12]

a0[:,0:11]=data_tran_While

w0 = np.random.random((11,N0)) # 9 inputs x 6 nodes
b0 = np.random.random((1,N0)) # 1 bias x 6 nodes
wb0=np.concatenate((w0, b0), axis=0)

a1=np.ones((len(data_tran_While),N0+1))
w1 = np.random.random((N0,N1)) # 6 inputs x 3 nodes
b1 = np.random.random((1,N1)) # 1 bias x 3 nodes
wb1=np.concatenate((w1, b1), axis=0)

a2=np.ones((len(data_tran_While),N1+1))
w2 = np.random.random((N1,N2)) # 3 input * 1 node
b2 = np.random.random((1,N2)) # 1 bias * 1 node
wb2=np.concatenate((w2, b2), axis=0)

a3 = np.ones((len(data_tran_While),N2))

for i in range(200000):
    #forward x=a0, a2=yp
    z0=np.dot(a0,wb0)
    a1[:,:-1]=g(z0)
    #a1=np.concatenate([g(z0),np.ones([a0.shape[0],1])], axis=1)
    
    z1=np.dot(a1,wb1)
    a2[:,:-1]=g(z1)
   
    z2=np.dot(a2,wb2)
    a3=g(z2)
    
    #backward
    
    d_a3=aws_tran_While-a3
    d_z2=d_a3*grad_g(a3)
    wb2 += eta*np.dot(a2.T,d_z2)
    
    d_a2=np.dot(d_z2,wb2.T)
    d_z1=d_a2*grad_g(a2)
    wb1 += eta*np.dot(a1.T,d_z1)[:,:-1]

    d_a1=np.dot(d_z1[:,:-1],wb1.T)
    d_z0=d_a1*grad_g(a1)
    wb0 += eta*np.dot(a0.T,d_z0[:,:-1])


MAE = (np.sum(np.absolute(d_a3)))/ len(d_a3)
print("MAE: {}".format(MAE))

MSE=0.5*np.sum(np.square(d_a3))
print("MSE: {}".format(MSE))

RMSE= np.sum(np.power(d_a3,2)) / len(d_a3)
print("RMSE: {}".format(RMSE))

print("========== TEST =============")     
a0=np.ones((len(data_test_While),12)) #[Nx10]

a0[:,0:11] = data_test_While 

a1=np.ones((len(data_test_While),N0+1))
a2=np.ones((len(data_test_While),N1+1))

#forward x=a0, a2=yp
z0=np.dot(a0,wb0)
a1[:,:-1]=g(z0)
#a1=np.concatenate([g(z0),np.ones([a0.shape[0],1])], axis=1)
    
z1=np.dot(a1,wb1)
a2[:,:-1]=g(z1)
   
z2=np.dot(a2,wb2)
a3=g(z2)

    
d_a3=aws_test_While-a3
MAE = (np.sum(np.absolute(d_a3)))/ len(d_a3)
print("MAE: {}".format(MAE))

MSE=0.5*np.sum(np.square(d_a3))
print("MSE: {}".format(MSE))

RMSE= np.sum(np.power(d_a3,2)) / len(d_a3)
print("RMSE: {}".format(RMSE))

MAE: 0.5213708626896013
MSE: 189.51388893494448
RMSE: 0.29634697253314224
========== TEST =============
MAE: 0.518750000046875
MSE: 46.80555556877778
RMSE: 0.2925347223048611
